# To Do list: 

In [193]:
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt
from collections import OrderedDict

In [194]:
def pmw2(workload, x, T, eps=0.01, k=0, show_messages=True, to_return='pd', analyst_labels = [],
       show_plot=False, show_failure_step=True):
    """
    Implement Private Multiplicative Weights Mechanism (PMW) on a workload of
    linear queries. New arguments to allow for optimizing the amount of
    privacy budget used in each step.
    
    to_return argument determines what the function will return. 
        - if 'pd', pmw() returns pandas df with test data for each 
        query (query, d_t_hat, updated, algo_ans, real_ans, abs_error, 
        rel_error). 
        - if 'update_count', pmw() returns the update count for the total
        amount of queries

    - W = workload of queries (M x k numpy array)
    - x = true database (M x 1 numpy array)
    - eps = privacy budget
    - k = number of update steps
    - T  = update threshold
    """ 
    
    # initialize constants
    m = x.size  # database len
    n = x.sum()
    eta = (math.log(m, np.e) ** (1 / 4)) / (math.sqrt(n))
    # Set k to be the desired number of update rounds
    if k==0: # essentially, if k hasn't been changed from its default value, use the length of the workload
        k = len(workload)  # num of queries
    delta = 1 / (n * math.log(n, np.e))
    x_norm = x / np.sum(x)
    # synthetic databases at time 0 (prior to any queries)
    y_t = np.ones(m) / m
    x_t = np.ones(m) / m

    # append to list of databases y_t and x_t
    x_list = [x_t]
    
    update_list = []
    update_count = 0
    pmw_answers = []
    update_times = []
    d_t_hat_list = []
    
    failure_mode = False # if reaches failure, failure mode because true and only runs lazy rounds
    
    def lazy_round():
        update_list.append('no')
        pmw_answers.append(np.dot(query, x_list[time]))
        x_list.append(x_list[time].round(3))
    
    #inititate first instance of SVT with half the budget and k updates
    #will be reset in the main loop
    SVTtrigger = False 
    SVTepsilon1 =((eps/2)/2)
    SVTepsilon2 = ((eps/2)/2)
    rho = np.random.laplace(loc=0, scale=(1/SVTepsilon1), size=1)[0]
    # iterate through time 
    for time, query in enumerate(workload):
        # Do one round of sparce vector technique 
        
        # compute noisy answer by adding Laplacian noise
        a_t = np.random.laplace(loc=0, scale=(2*k/SVTepsilon2), size=1)[0]
        a_t_hat = (np.dot(query, x_norm)*n ) + a_t

        # difference between noisy and maintained histogram answer
        d_t_hat = a_t_hat - (n*np.dot(query, x_list[time]))
        
        # lazy round: use maintained histogram to answer the query
        if (abs(d_t_hat) <= T + rho) or failure_mode:
            d_t_hat_list.append(d_t_hat)
            lazy_round()
            continue

        # update round: update histogram and return noisy answer
        else:
            update_times.append(time)
            #make a new noisy query answer using some of the leftover budget
            a_t = np.random.laplace(loc=0, scale=(2*k/eps), size=1)[0]
            a_t_hat = (np.dot(query, x_norm)*n ) + a_t
            d_t_hat = a_t_hat - (n*np.dot(query, x_list[time]))
            d_t_hat_list.append(d_t_hat)
            
            # step a
            if d_t_hat < 0:
                r_t = query
            else:
                r_t = np.ones(m) - query
            for i, v in enumerate(y_t):
                #y_t[i] = x_list[time][i] * math.exp(-eta * r_t[i])
                y_t[i] = x_list[time][i] * math.exp((d_t_hat/(2*n)) * query[i])
            
            #print(d_t_hat)
            #print(query)
            #print(x_list[time])
            # step b
            x_t = y_t / np.sum(y_t)
            update_count = update_list.count('yes')
            
            if update_count >= k: # threshold for num updates is reached, enter failure_mode
                failure_mode = True
                if show_failure_step:
                    print(f'Failure mode reached at query number {time}: {query}')
                lazy_round()
            else: # threshold for num updates is not reached yet
                x_list.append(x_t.round(3))
                update_list.append('yes') # increment number of updates counter
                pmw_answers.append(a_t_hat / np.sum(x))
                
    update_count = update_list.count('yes')      

    # calculate error
    real_ans = np.matmul(workload, x_norm)
    abs_error = np.abs(pmw_answers - real_ans)
    rel_error = np.abs(abs_error / np.where(real_ans == 0, 0.000001,
                                                real_ans))
    
    def print_outputs():
        np.set_printoptions(suppress=True)
        """Print inputes/outputs to analyze each query"""
        print(f'Original database: {x}\n')
        print(f'Normalized database: {x_norm}\n')
        print(f'Updated Database = {x_t}\n')
        print(f'Update Count = {update_count}\n')
        print(f'{T=}\n')
        print(f'Error Scale Query Answer= {2*((2*k/eps)**2)}\n')
        print(f'Error Scale SVT= {2*((2*k/SVTepsilon2)**2)}\n')
        print(f'Update Parameter Scale = {eta}\n')
        print(f'{delta=}\n')
    
    if show_messages:
        print_outputs()
        
    if show_plot: 
        plt.xticks(range(0, len(workload), 5))
        plt.title('Error across queries:')
        #rel_line, = plt.plot(rel_error, label='Relative Error')
        abs_line, = plt.plot(abs_error, label='Absolute Error')
        for xc in update_times:
            plt.axvline(x=xc, color='red', label='Update Times', linestyle='dashed')
        plt.legend(handles=[abs_line]) #,rel_line])
        
    if to_return == "pd":
        x_list.pop(0).tolist() # remove the first synthetic database to keep length of lists consistent-x_list[t] represents the synthetic database at the end of time t
        
        
        d = {
            'algo_ans': pmw_answers,
            'real_ans': real_ans.tolist(),
            'queries': workload.tolist(), 
            'updated': update_list,
            'abs_error': abs_error,               
#            'rel_error': rel_error,
            'synthetic database': x_list,
            'analyst': analyst_labels,
            'd_t_hat': d_t_hat_list, 

             }
        test_data = pd.DataFrame(data=d)
        test_data = test_data.round(3)
        return test_data
    
    if to_return == "error":
        #unique_analysts = list(OrderedDict.fromkeys(analyst_labels))
        
        d = {'analyst': analyst_labels,
             'abs_error': abs_error,               
             'rel_error': rel_error,}
        data = pd.DataFrame(data=d)
        data = data.round(3)
        
        analyst_error = {}
        for analyst in list(set(analyst_labels)):
            analyst_error[analyst] = data[data.analyst==analyst]['abs_error'].sum()
        return analyst_error

        

### Initializing workloads and databases

In [3]:
x_small = np.array([20, 160, 20, 20, 20, 180, 20, 20])
normalized = x_small / x_small.sum()
m = x_small.size  # database len
n = x_small.sum()
#print(f'the threshold for failure is {n * math.log(m, np.e) ** (1 / 2)}')

random_array = np.random.randint(2, size=(500,4))
zero_array = np.zeros((500,4))
alice = np.hstack((random_array, zero_array))
bob = np.hstack((zero_array, random_array))

We've shown that Alice can use up all the privacy budget. 

Next, let's take some of the basic adaptations of pmw and see if we can get the same effects to happen or a similar effect were people get similar results indpenedently

### To Dos: 

Instead of having a communal pool of update steps we can do, split the amount of update steps evenly across all analysts. If your update steps are not a multiple of the analysts, you can change the number of update steps.  

Look for the same kind of violation where either Bob or Alice will have more overall error in the joint case as opposed to the independent case.

In the independent state, you can use the existing pmw algorihtm.

Suggestion: When you split it to their individual settings, give them the same amount of update steps that they would've gotten in the group. 

Individually - 1 ep, 5 k. together - 2 ep, 10k. 

### Experiments: 
Try original pmw, adapted w equal update steps pmw, individual: 
1. [done] A and B query disjoint sections
2. A queries the entire dataset except for last index. B queries entire dataset, last index inclusive. 
3. A asks singleton, B asks all range queries
4. [if time] Is total error the right metric? Context: You might run into a case where you get flat error across all of your queries. You may get a lot worse error on one specific query, but the overall error is better. Now Alice and Bob care about the same data. Alice eats all the budget (entire database except for last). Bob cares about entire database, but his queries about the last index has higher error. 
    
    
Scenarios to look for: 
1. Regular pmw performs poorly, but adapted pmw works well. (probably in disjoint setting)
2. Scenarios where regular AND adapted pmw perform poorly

## Scenario 1: Disjoint

In [336]:
# initialize
s1random_array = np.random.randint(2, size=(25,4))
s1zero_array = np.zeros((25,4))
s1alice = np.hstack((s1random_array, s1zero_array))
s1bob = np.hstack((s1zero_array, s1random_array))

s1combined_list = []
s1individual_list = []

for i in range(100):
    # combined
    s1combined = pmw2(workload=np.vstack((s1alice, s1bob)), x=x_small, eps=2, T=40, k=10,  
                                 analyst_labels=['Alice'] * 25 + ['Bob'] * 25, 
                                 to_return='error', show_plot=False, show_messages=False, show_failure_step=False)
    s1combined_list.append(s1combined)
    
    # individual
    s1a = pmw2(workload=s1alice, x=x_small, eps=1, T=40, k=5,
               analyst_labels=['Alice'] * 25, 
               to_return='error', show_plot=False, show_messages=False, show_failure_step=False)
    s1b = pmw2(workload=s1bob, x=x_small, eps=1, T=40, k=5,
               analyst_labels=['Bob'] * 25, 
               to_return='error', show_plot=False, show_messages=False, show_failure_step=False)
    s1individual = {**s1a, **s1b}
    s1individual_list.append(s1individual)
    
# find mean over multiple trials
s1combined_average = dict(pd.DataFrame(s1combined_list).mean())
s1individual_average = dict(pd.DataFrame(s1individual_list).mean())


d = {'combined': s1combined_average, 'individual': s1individual_average}
df = pd.DataFrame(data=d).sort_index()
display(df)

,combined,individual
Alice,1.27986,1.89754
Bob,3.14006,2.12520


Bob always receives more error in the combined setting than in the individual setting.

Questions: 

1. Why does Bob always receives more error than Alice in the individual setting if their queries are not related? I would expect instead that their individual error would be very similar.
2. Alice receives substantially more error in the individual setting than the combined setting. Is this supposed to happen? Although she receives less update steps in the individual setting, shouldn't the smaller epsilon value in the inidivual case counteract this effect?


# Scenario 2: Last Index

In [366]:
# combined
s2alice = np.hstack((np.random.randint(2, size=(25,7)), np.zeros((25,1))))
s2bob = np.random.randint(2, size=(25,8))

s2combined = pmw2(workload=np.vstack((s2alice, s2bob)), x=x_small, eps=1, T=40, k=10,  
                             analyst_labels=['Alice'] * 25 + ['Bob'] * 25, 
                             to_return='error', show_plot=False, show_messages=False, show_failure_step=False)
s2combined

{'Bob': 3.166, 'Alice': 2.65}

In [367]:
# individual
s1a = pmw2(workload=s2alice, x=x_small, eps=1, T=40, k=10,
           analyst_labels=['Alice'] * 25, 
           to_return='error', show_plot=False, show_messages=False, show_failure_step=False)
s1b = pmw2(workload=s2bob, x=x_small, eps=1, T=40, k=10,
           analyst_labels=['Bob'] * 25, 
           to_return='error', show_plot=False, show_messages=False, show_failure_step=False)
s1individual = {**s1a, **s1b}
s1individual

{'Alice': 2.3080000000000003, 'Bob': 2.47}

Bob always receives more error in the combined setting than in the individual setting.

# Scenario 3: Incompatible Queries

In [ ]:
s3alice = 

# Results:
- Alice's total error individually:
- Bob's total error individually: 
- Alice's total error in collective: 
- Bob's total error in collective: 

# Conclusion: 

# Follow-ups: